In [ ]:
# %pip install matplotlib pandas

In [ ]:
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Get the name of latest CSV file
#csv_filename = sorted(glob.glob("./result-*.csv"))[-1]


# Use fixed CSV filename
#csv_filename = "./result-20241121-001238.csv"
csv_filename = "./result-20241121-161421.csv"


print(csv_filename)

In [ ]:

df = pd.read_csv(csv_filename)
df.head(10)

In [ ]:
# Add integer version of transfer_size column
def size_str_to_int(s):
    if s.endswith("K"):
        i = int(s[:-1])
    elif s.endswith("M"):
        i = int(s[:-1]) * 1024
    elif s.endswith("G"):
        i = int(s[:-1]) * 1024 * 1024
    elif s.endswith("T"):
        i = int(s[:-1]) * 1024 * 1024 * 1024
    else:
        assert False
    return i

df["transfer_size_int"] = df["transfer_size"].map(size_str_to_int)

df.head()

In [ ]:
df.dtypes

In [ ]:
# Common functions for plot

def format_num_processes(x, _):
    return "%d" % (x)

def format_filesize(x, _):
    if x<0:
        return str(x)
    for x_unit in ['KB', 'MB', 'GB', 'TB']:
        if x < 1024:
            return "%d%s" % (x, x_unit)
        x /= 1024.0

def format_bandwidth(y, _):
    return "%dMB/s" % (y)


In [ ]:
# test-1: granularity x 3 variations, number of processes x 9 variations

xaxis_ticks = []
for i in range(9):
    xaxis_ticks.append( 1 * (2**i) )

def plot_graph( title, condition, values ):

    df2 = df[condition]
    df3 = df2.pivot(index="num_processes", columns=["filesystem_type"], values=values)
    ax = df3.plot(title=title, logx=True, figsize=(6, 4), rot=0)
    ax.set_xlabel("num processes (log scale)")
    ax.xaxis.set_ticks(xaxis_ticks)
    ax.xaxis.set_major_formatter(format_num_processes)
    ax.yaxis.set_major_formatter(format_bandwidth)

if 0:
    plot_graph( "Read bandwidth 4K", df["transfer_size"]=="4K", ["read_bw_mean"] )
    plot_graph( "Write bandwidth 4K", df["transfer_size"]=="4K", ["write_bw_mean"] )

    plot_graph( "Read bandwidth 1M", df["transfer_size"]=="1M", ["read_bw_mean"] )
    plot_graph( "Write bandwidth 1M", df["transfer_size"]=="1M", ["write_bw_mean"] )

    plot_graph( "Read bandwidth 256M", df["transfer_size"]=="256M", ["read_bw_mean"] )
    plot_graph( "Write bandwidth 256M", df["transfer_size"]=="256M", ["write_bw_mean"] )

In [ ]:
# test-2: number of processes = 32, granularity x 17, variations

xaxis_ticks = []
for i in range(16):
    xaxis_ticks.append( 4 * (2**i) )

def plot_graph( title, condition, values ):

    df2 = df[condition]
    df3 = df2.pivot(index="transfer_size_int", columns=["filesystem_type"], values=values)
    ax = df3.plot(title=title, logx=True, figsize=(6, 4), rot=-45)
    ax.xaxis.set_ticks(xaxis_ticks)
    ax.xaxis.set_major_formatter(format_filesize)
    ax.yaxis.set_major_formatter(format_bandwidth)

plot_graph( "Read bandwidth 8 processes", df["num_processes"]==8, ["read_bw_mean"] )
plot_graph( "Write bandwidth 8 processes", df["num_processes"]==8, ["write_bw_mean"] )

plot_graph( "Read bandwidth 32 processes", df["num_processes"]==32, ["read_bw_mean"] )
plot_graph( "Write bandwidth 32 processes", df["num_processes"]==32, ["write_bw_mean"] )

plot_graph( "Read bandwidth 128 processes", df["num_processes"]==128, ["read_bw_mean"] )
plot_graph( "Write bandwidth 128 processes", df["num_processes"]==128, ["write_bw_mean"] )
